In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import sys
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

from importlib import reload
sys.path.append("D:/DST/")
import cathay_db as db
import utils as ut
import financial_statement as fs

reload(ut)
reload(fs)
import matplotlib.pyplot as plt
import datetime
import os
import xgboost as xgb
sys.path.append("./src")

from toolbox import print_progress_bar

# set max display rows
pd.set_option("display.max_rows", 200)

# set max display columns
pd.set_option("display.max_columns", 100)

# Set the float format to display without scientific notation
pd.options.display.float_format = '{:.4f}'.format

In [2]:
# select model 
model_folder_name = '20230711_171428'
# model_folder_name = '20230711_110457'
# model_folder_name = '20230707_140403'
# model_folder_name = '20230706_141045'

# load setting from data/model/model_folder_name/setting.json
setting = ut.load_json(f'./data/model/{model_folder_name}/setting.json')

print(setting.keys())
print(setting['factor_columns'])

dict_keys(['start_index', 'rebalance_date_lst', 'factor_columns', 'target_cols', 'model_save_path', 'param_search'])
['asset_qoq', 'asset_yoy', 'ni_qoq', 'ni_yoy', 'roe', 'roe_yoy', 'roe_4q_sum', 'roe_4q_sum_yoy', 'tobins_q', 'ocf / asset', '20_d_return', '40_d_return', '60_d_return', 'dividend_1Y_sum_yield', 'dividend_2Y_sum_yield', 'dividend_3Y_sum_yield', 'last_dividend_yield']


In [3]:
# create a list for model json file in model/model_folder_name
model_list = []
for model in os.listdir(f'./models/{model_folder_name}'):
    if model.endswith('.json'):
        # append the model path to model_list
        model_list.append(f'./models/{model_folder_name}/{model}')
# sort model_list
model_list.sort()

In [4]:
# load df_factor from data/model/model_folder_name/df_factor_all.feather
df_factor_all = pd.read_feather(f'./data/model/{model_folder_name}/df_factor_all.feather')

In [5]:
# load df_factor from data/model/model_folder_name/df_factor_all.feather
df_factor_all = pd.read_feather(f'./data/model/{model_folder_name}/df_factor_all.feather')

# create a dataframe for model mapping
df_model_mapping = pd.DataFrame()
df_model_mapping['rebalance_date'] = setting['rebalance_date_lst'][setting['start_index'] + 0:]
df_model_mapping['model_path'] = model_list
# convert rebalance_date to datetime
df_model_mapping['rebalance_date'] = pd.to_datetime(df_model_mapping['rebalance_date'])

# merge the df_model_mapping and df_factor_all
df_factor_all = pd.merge(df_factor_all, df_model_mapping, left_on='date', right_on='rebalance_date', how='left')
df_factor_all = df_factor_all.sort_values(['date','ticker']).reset_index(drop=True)
df_factor_all['model_path'].fillna(method='ffill', inplace=True)
df_factor_all['rebalance_date'].fillna(method='ffill', inplace=True)
df_factor_all.dropna(subset=['model_path'], inplace=True)

df_factor_all = df_factor_all.reset_index(drop=True)
df_factor_all.to_feather(f'./data/model/{model_folder_name}/df_factor_all_select.feather')

In [6]:
factor_columns = setting['factor_columns']

In [7]:
model_list = df_factor_all['model_path'].unique()
reb_lst = df_factor_all['rebalance_date'].unique()

In [8]:
df_predict_all = pd.DataFrame([])
df_feature_importance_all = pd.DataFrame([])
for i, model_path in enumerate(model_list):
    model = xgb.Booster()
    model.load_model(model_path)
    try:
        df_feature_importance = pd.DataFrame(model.get_score(importance_type='gain').items(), columns=['feature','importance'])
        df_feature_importance.sort_values('importance', ascending=False, inplace=True)
        df_feature_importance['date'] = reb_lst[i]
        df_feature_importance_all = pd.concat([df_feature_importance_all, df_feature_importance], axis=0)

        df_predict = df_factor_all[df_factor_all['rebalance_date'] == reb_lst[i]].reset_index(drop=True).copy()
        df_predict['predict'] = model.predict(xgb.DMatrix(df_predict[factor_columns]))
        df_predict_all = pd.concat([df_predict_all, df_predict], axis=0)
    except:
        print(reb_lst)

In [9]:
df_feature_importance_des = df_feature_importance_all.groupby('feature')['importance'].describe().sort_values('mean', ascending=False).reset_index()
df_feature_importance_des['IR'] = df_feature_importance_des['50%'] / df_feature_importance_des['std']
# rank IR
df_feature_importance_des['mean_rank'] = df_feature_importance_des['mean'].rank(ascending=False)
df_feature_importance_des['IR_rank'] = df_feature_importance_des['IR'].rank(ascending=False)

df_feature_importance_des.sort_values('mean', ascending=False, inplace=True)
df_feature_importance_des

,feature,count,mean,std,min,25%,50%,75%,max,IR,mean_rank,IR_rank
0,tobins_q,72.0000,3.0535,1.1163,0.8835,2.2633,2.9778,3.7237,5.5394,2.6676,1.0000,11.0000
1,60_d_return,72.0000,3.0112,1.0386,1.4497,2.2427,2.7840,3.3775,6.9728,2.6805,2.0000,9.0000
2,roe,72.0000,2.9327,1.4612,1.6470,2.1248,2.4290,3.4381,9.4194,1.6624,3.0000,14.0000
3,dividend_2Y_sum_yield,72.0000,2.7259,0.9061,1.2172,2.0211,2.5463,3.4872,6.9824,2.8102,4.0000,7.0000
4,20_d_return,72.0000,2.6230,0.8052,1.3867,2.0338,2.3806,3.1212,4.9550,2.9564,5.0000,6.0000
5,roe_yoy,72.0000,2.5998,1.5154,0.8569,1.7512,2.1892,2.8205,10.0509,1.4446,6.0000,15.0000
6,roe_4q_sum,72.0000,2.5305,0.6262,1.6567,2.0454,2.3941,2.8313,4.3600,3.8233,7.0000,1.0000
7,40_d_return,72.0000,2.5195,0.6138,1.5593,2.0059,2.3051,3.1543,3.9903,3.7555,8.0000,2.0000
8,ni_qoq,72.0000,2.3069,1.0955,0.9023,1.5381,2.0929,2.6276,7.0696,1.9105,9.0000,12.0000
9,ocf / asset,72.0000,2.2684,0.6708,0.9750,1.7881,2.1061,2.5347,4.2761,3.1398,10.0000,4.0000


In [10]:
i = 0

df_predict_describe_all = pd.DataFrame([])

for i in range(len(reb_lst)):
    df_predict = df_predict_all[df_predict_all['rebalance_date'] == reb_lst[i]].reset_index(drop=True).copy()
    # calc predict qcut
    df_predict['predict_qcut'] = pd.qcut(df_predict['predict'], 10, labels=False, duplicates='drop')
    df_predict_describe = df_predict.groupby('predict_qcut')['y'].describe().reset_index()
    df_predict_describe['date'] = reb_lst[i]
    df_predict_describe_all = pd.concat([df_predict_describe_all, df_predict_describe], axis=0)

In [11]:
df_predict_describe_all.groupby('predict_qcut')['mean'].describe().sort_values('mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
predict_qcut,,,,,,,,
9,72.0000,0.0563,0.1165,-0.2628,0.0067,0.0577,0.1046,0.4863
8,72.0000,0.0492,0.1112,-0.2942,0.0081,0.0506,0.0929,0.4738
7,72.0000,0.0466,0.1083,-0.3053,0.0013,0.0481,0.0919,0.4574
6,72.0000,0.0447,0.1061,-0.3008,0.0031,0.0449,0.0869,0.4600
5,72.0000,0.0431,0.1034,-0.3031,-0.0051,0.0451,0.0866,0.4434
4,72.0000,0.0398,0.1042,-0.2984,-0.0055,0.0386,0.0805,0.4873
3,72.0000,0.0379,0.1028,-0.3040,-0.0022,0.0376,0.0820,0.4605
2,72.0000,0.0338,0.1034,-0.3146,-0.0029,0.0326,0.0792,0.4578
1,72.0000,0.0338,0.1086,-0.3486,-0.0107,0.0303,0.0798,0.4683


In [12]:
df_predict_all = df_predict_all[df_predict_all['date'].isin(reb_lst)]
# df_predict_all['predict_group'] = df_predict_all.groupby('rebalance_date',group_keys=False)['predict'].apply(lambda x :pd.qcut(x, 5, labels=False))
df_predict_all['predict_rank'] = df_predict_all.groupby('rebalance_date',group_keys=False)['predict'].rank(ascending=False)
df_select = df_predict_all[df_predict_all['predict_rank']<=50]
df_select_return = df_select.groupby('date')['y'].mean()


def calculate_cagr(df):
    # Add 1 to all the quarterly returns
    df_plus_one = df + 1
    # Calculate the cumulative product of the returns
    cum_product = df_plus_one.cumprod()
    # Get the total number of years
    num_years = len(df) / 4
    # Calculate CAGR
    cagr = (cum_product.iloc[-1])**(1/num_years) - 1
    return cagr


In [13]:
print(f"SR: {((df_select_return + 1).prod()**(4/len(df_select_return)) -1 ) / (df_select_return.std() * np.sqrt(4)):.2f}")
# Assuming your DataFrame is named df_returns
cagr = calculate_cagr(df_select_return)
print(f"CAGR: {cagr:.2%}")
print(f"Std: {(df_select_return.std() * np.sqrt(4)):.2%}")

SR: 0.80
CAGR: 22.20%
Std: 27.87%


In [14]:
# follow code is not used because the top stocks is select by last dividend yield and predict_rank, 
# the result will fulfill the requirement (dividend yield >= 5% and predict_rank <= certain rank)

# dividend_min = 0.05
# df_dividend_filter = df_predict_all[df_predict_all['last_dividend_yield'] >= dividend_min].copy()
# df_predict_filter = df_predict_all[df_predict_all['predict_rank'] <= 300]
# df_dividend_select = pd.concat([df_dividend_filter, df_predict_filter]).drop_duplicates().reset_index(drop=True)
# df_dividend_select.sort_values(['date','ticker'], inplace=True)

## Consider Liquidity

In [15]:
# df_price_liquidity_flag.to_feather('./data/df_price_liquidity_flag.feather')
df_price_liquidity_flag = pd.read_feather('./data/df_price_liquidity_flag.feather')
df_price_liquidity_flag = df_price_liquidity_flag[['日期','股票代號','flag_all']]
df_price_liquidity_flag.columns = ['date','ticker','flag_all']


df_predict_all = pd.merge(df_predict_all, df_price_liquidity_flag, on=['date','ticker'], how='left')
df_select_pool = df_predict_all[df_predict_all['flag_all'] != 1].reset_index(drop=True).copy()


In [16]:
def top_n(df, n=5, columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True]):
    return df.sort_values(by=columns, ascending=ascending).head(n)

df_select_pool = df_select_pool.reset_index(drop=True)
df_select_pool_top_50 = df_select_pool.groupby('date').apply(top_n, n=50, columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True])

df_select_pool_top_50.reset_index(drop=True, inplace=True)
df_select_pool_top_50_return = df_select_pool_top_50.groupby('date')['y'].mean()
print("不做選股Buffer")
print(f"SR: {((df_select_pool_top_50_return + 1).prod()**(4/len(df_select_pool_top_50_return)) -1 ) / (df_select_pool_top_50_return.std() * np.sqrt(4)):.2f}")
# Assuming your DataFrame is named df_returns
cagr = calculate_cagr(df_select_pool_top_50_return)
print(f"CAGR: {cagr:.2%}")
print(f"Std: {(df_select_pool_top_50_return.std() * np.sqrt(4)):.2%}")


不做選股Buffer
SR: 0.62
CAGR: 16.41%
Std: 26.30%


In [17]:
df_select_pool.to_feather(f'./data/model/{model_folder_name}/df_select_pool.feather')

## Consider Turnover

In [21]:
apply_buffer_index = 0
base_portfolio_number = 50
buffer_portfolio_number = 150

In [22]:
df_select_pool = df_select_pool.reset_index(drop=True)
# get the base portfolio by base_portfolio_number 
df_select_pool_base = df_select_pool.groupby('date').apply(top_n, n = base_portfolio_number, columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True]).reset_index(drop=True)
# get the buffer portfolio by buffer_portfolio_number
df_select_pool_buffer = df_select_pool.groupby('date').apply(top_n, n = buffer_portfolio_number, columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True]).reset_index(drop=True)
df_select_pool_buffer.reset_index(drop=True, inplace=True)


apply_buffer_date = np.sort(reb_lst)[apply_buffer_index]
# string format
apply_buffer_date = pd.to_datetime(apply_buffer_date).strftime('%Y-%m-%d')
print(f'自 {apply_buffer_date} 開始使用 buffer 建構投組')

df_buffer_result = pd.DataFrame([])
# concat the data that date is smaller than apply_buffer_date to df_buffer_result
df_buffer_result = pd.concat([df_buffer_result, df_select_pool_base[df_select_pool_base['date'] < apply_buffer_date]], axis=0)
# print the max date in df_buffer_result
print(f"df_buffer_result['date'].max(): {df_buffer_result['date'].max()}")

for i in range(apply_buffer_index, len(reb_lst)):
    # get the slice data by reb_lst[i] in df_select_pool_buffer
    df_select_pool_buffer_temp = df_select_pool_buffer[df_select_pool_buffer['rebalance_date'] == reb_lst[i]].reset_index(drop=True)
    # get the slice data by reb_lst[i-1] in df_buffer_result, [i-1] is because we need to get the last portfolio
    df_buffer_result_temp = df_buffer_result[df_buffer_result['rebalance_date'] == reb_lst[i - 1]].reset_index(drop=True)

    # prepare the dataframe for finding the joint ticker
    df_new_temp = df_select_pool_buffer_temp[['date','ticker']].copy()
    df_new_temp['buffer'] = 1
    df_old_temp = df_buffer_result_temp[['date','ticker']].copy()
    df_old_temp['last_portfolio'] = 1

    # merge the df_old_temp and df_new_temp
    df_joint_temp = pd.merge(df_old_temp[['ticker','last_portfolio']], df_new_temp[['ticker','buffer']], on=['ticker'], how='outer')
    df_joint_temp['buffer'].fillna(0, inplace=True)
    df_joint_temp['last_portfolio'].fillna(0, inplace=True)
    df_joint_temp['total'] = df_joint_temp['buffer'] + df_joint_temp['last_portfolio']
    df_joint_temp = df_joint_temp[df_joint_temp['total'] == 2]

    # get the joint ticker
    joint_ticker = df_joint_temp['ticker'].unique()

    # get the priority portfolio by joint_ticker
    df_priority = df_select_pool_buffer_temp[df_select_pool_buffer_temp['ticker'].isin(joint_ticker)]
    # get the triviality portfolio by EXCLUDING joint_ticker
    df_triviality = df_select_pool_buffer_temp[df_select_pool_buffer_temp['ticker'].isin(joint_ticker) == False]
    # then find the (top n - len(df_priority)) number of stocks triviality portfolio
    df_triviality = df_triviality.groupby('date').apply(top_n, n = base_portfolio_number - len(df_priority), columns=['last_dividend_yield', 'predict_rank'], ascending=[False, True]).reset_index(drop=True)
    
    # concat the df_priority and df_triviality
    df_buffer_result_temp = pd.concat([df_priority, df_triviality], axis=0)

    # concat the df_buffer_result_temp to df_buffer_result
    df_buffer_result = pd.concat([df_buffer_result, df_buffer_result_temp], axis=0)

df_buffer_result.reset_index(drop=True, inplace=True)
df_buffer_result_return = df_buffer_result.groupby('date')['y'].mean()

df_buffer_result.to_feather(f'./data/model/{model_folder_name}/df_buffer_result_{apply_buffer_index}_{base_portfolio_number}_{buffer_portfolio_number}.feather')
print(f"SR: {((df_buffer_result_return + 1).prod()**(4/len(df_buffer_result_return)) -1 ) / (df_buffer_result_return.std() * np.sqrt(4)):.2f}")
# Assuming your DataFrame is named df_returns
cagr = calculate_cagr(df_buffer_result_return)
print(f"CAGR: {cagr:.2%}")
print(f"Std: {(df_buffer_result_return.std() * np.sqrt(4)):.2%}")

自 2005-03-15 開始使用 buffer 建構投組
df_buffer_result['date'].max(): NaT
SR: 0.60
CAGR: 15.30%
Std: 25.32%
